# 探索 OpenAI V1 功能

在 11.06.23，OpenAI 发布了许多新功能，并将他们的 Python SDK 升级到了 1.0.0 版本。本笔记本展示了新功能以及如何在 LangChain 中使用它们。

In [ ]:
# 需要安装 openai>=1.1.0, langchain>=0.0.335, langchain-experimental>=0.0.39
!pip install -U openai langchain langchain-experimental
这段代码是用来安装指定版本的 openai、langchain 和 langchain-experimental 包的。

In [1]:
# 导入所需模块
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

## [视觉](https://platform.openai.com/docs/guides/vision)

OpenAI发布了多模态模型，可以将文本序列和图像作为输入。

In [2]:
# 导入ChatOpenAI类
from openai import ChatOpenAI, HumanMessage

# 创建ChatOpenAI实例，指定模型为"gpt-4-vision-preview"，最大token数为256
chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=256)

# 调用invoke方法，传入HumanMessage实例作为参数
chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": "What is this image showing"},  # 询问图片展示的内容
                {
                    "type": "image_url",  # 指定消息类型为图片链接
                    "image_url": {
                        "url": "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/static/img/langchain_stack.png",  # 图片链接
                        "detail": "auto",  # 图片细节自动处理
                    },
                },
            ]
        )
    ]
)

AIMessage(content='The image appears to be a diagram representing the architecture or components of a software system or framework related to language processing, possibly named LangChain or associated with a project or product called LangChain, based on the prominent appearance of that term. The diagram is organized into several layers or aspects, each containing various elements or modules:\n\n1. **Protocol**: This may be the foundational layer, which includes "LCEL" and terms like parallelization, fallbacks, tracing, batching, streaming, async, and composition. These seem related to communication and execution protocols for the system.\n\n2. **Integrations Components**: This layer includes "Model I/O" with elements such as the model, output parser, prompt, and example selector. It also has a "Retrieval" section with a document loader, retriever, embedding model, vector store, and text splitter. Lastly, there\'s an "Agent Tooling" section. These components likely deal with the intera

## [OpenAI助手](https://platform.openai.com/docs/assistants/overview)

> Assistants API允许您在自己的应用程序中构建AI助手。助手具有指令，并可以利用模型、工具和知识来回答用户的查询。助手API目前支持三种类型的工具：代码解释器、检索和函数调用。

您可以使用OpenAI工具或自定义工具与OpenAI助手进行交互。当仅使用OpenAI工具时，您可以直接调用助手并获得最终答案。当使用自定义工具时，您可以使用内置的AgentExecutor运行助手和工具执行循环，或者轻松编写自己的执行器。

下面我们展示了与助手交互的不同方式。作为一个简单的例子，让我们构建一个能够编写和运行代码的数学导师。

### 仅使用OpenAI工具



In [1]:
# 导入OpenAIAssistantRunnable类
from langchain.agents.openai_assistant import OpenAIAssistantRunnable

In [2]:
# 创建一个OpenAIAssistantRunnable对象，用于创建一个助手
interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",  # 设置助手的名称为"langchain assistant"
    instructions="You are a personal math tutor. Write and run code to answer math questions.",  # 设置助手的指导说明
    tools=[{"type": "code_interpreter"}],  # 设置助手的工具为代码解释器
    model="gpt-4-1106-preview",  # 设置助手使用的模型为"gpt-4-1106-preview"
)

# 调用助手来处理输入内容"What's 10 - 4 raised to the 2.7"
output = interpreter_assistant.invoke({"content": "What's 10 - 4 raised to the 2.7"})

# 输出助手处理后的结果
output

[ThreadMessage(id='msg_g9OJv0rpPgnc3mHmocFv7OVd', assistant_id='asst_hTwZeNMMphxzSOqJ01uBMsJI', content=[MessageContentText(text=Text(annotations=[], value='The result of \\(10 - 4^{2.7}\\) is approximately \\(-32.224\\).'), type='text')], created_at=1699460600, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_nBIT7SiAwtUfSCTrQNSPLOfe', thread_id='thread_14n4GgXwxgNL0s30WJW5F6p0')]

### 作为一个拥有任意工具的LangChain代理

现在让我们使用我们自己的工具来重新创建这个功能。在这个例子中，我们将使用[E2B沙盒运行时工具](https://e2b.dev/docs?ref=landing-page-get-started)。

In [ ]:
# 导入所需的库
!pip install e2b duckduckgo-search

In [3]:
from langchain.tools import DuckDuckGoSearchRun, E2BDataAnalysisTool

tools = [E2BDataAnalysisTool(api_key="..."), DuckDuckGoSearchRun()]

In [4]:
# 导入所需的库
from openai import OpenAIAssistantRunnable

# 创建一个名为 "langchain assistant e2b tool" 的助手
# 指定助手的说明为 "You are a personal math tutor. Write and run code to answer math questions. You can also search the internet."
# 指定助手的工具为 tools
# 指定助手的模型为 "gpt-4-1106-preview"
# 将助手设置为代理模式
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions. You can also search the internet.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True,
)

#### 使用AgentExecutor

In [5]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({"content": "What's the weather in SF today divided by 2.7"})

{'content': "What's the weather in SF today divided by 2.7",
 'output': "The weather in San Francisco today is reported to have temperatures as high as 66 °F. To get the temperature divided by 2.7, we will calculate that:\n\n66 °F / 2.7 = 24.44 °F\n\nSo, when the high temperature of 66 °F is divided by 2.7, the result is approximately 24.44 °F. Please note that this doesn't have a meteorological meaning; it's purely a mathematical operation based on the given temperature."}

#### 自定义执行

In [6]:
# 创建一个OpenAIAssistantRunnable对象，代表一个可以运行的助手
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",  # 设置助手的名称为"langchain assistant e2b tool"
    instructions="You are a personal math tutor. Write and run code to answer math questions.",  # 设置助手的指示为"你是一个个人数学辅导老师。编写并运行代码来回答数学问题。"
    tools=tools,  # 设置助手的工具
    model="gpt-4-1106-preview",  # 设置助手使用的模型为"gpt-4-1106-preview"
    as_agent=True,  # 将助手设置为代理模式
)

In [7]:
from langchain_core.agents import AgentFinish


def execute_agent(agent, tools, input):
    # 创建一个工具映射字典，将工具名称映射到工具对象
    tool_map = {tool.name: tool for tool in tools}
    # 调用代理的invoke方法，并将输入作为参数传入
    response = agent.invoke(input)
    # 当响应不是AgentFinish类型时，循环执行以下代码块
    while not isinstance(response, AgentFinish):
        # 创建一个工具输出列表
        tool_outputs = []
        # 遍历响应中的每个动作
        for action in response:
            # 调用对应工具的invoke方法，并将动作的工具输入作为参数传入
            tool_output = tool_map[action.tool].invoke(action.tool_input)
            # 打印工具名称、工具输入、工具输出
            print(action.tool, action.tool_input, tool_output, end="\n\n")
            # 将工具输出和工具调用ID添加到工具输出列表中
            tool_outputs.append(
                {"output": tool_output, "tool_call_id": action.tool_call_id}
            )
        # 调用代理的invoke方法，并将工具输出、运行ID和线程ID作为参数传入
        response = agent.invoke(
            {
                "tool_outputs": tool_outputs,
                "run_id": action.run_id,
                "thread_id": action.thread_id,
            }
        )

    # 返回响应
    return response

In [8]:
# 调用execute_agent函数执行代理
response = execute_agent(agent, tools, {"content": content})

# 打印输出结果
print(response.return_values["output"])

e2b_data_analysis {'python_code': 'print(10 - 4 ** 2.7)'} {"stdout": "-32.22425314473263", "stderr": "", "artifacts": []}

\( 10 - 4^{2.7} \) is approximately \(-32.22425314473263\).


In [9]:
# 调用 execute_agent 函数，传入 agent, tools 和参数字典
next_response = execute_agent(
    agent, tools, {"content": "now add 17.241", "thread_id": response.thread_id}
)
# 打印 next_response 的 return_values 字典中的 "output" 值
print(next_response.return_values["output"])

e2b_data_analysis {'python_code': 'result = 10 - 4 ** 2.7\nprint(result + 17.241)'} {"stdout": "-14.983253144732629", "stderr": "", "artifacts": []}

When you add \( 17.241 \) to \( 10 - 4^{2.7} \), the result is approximately \( -14.98325314473263 \).


## [JSON 模式](https://platform.openai.com/docs/guides/text-generation/json-mode)

将模型限制为仅生成有效的 JSON。请注意，您必须包含一条系统消息，其中包含使用 JSON 的说明，以使此模式正常工作。

仅适用于某些模型。

In [ ]:
# 创建一个ChatOpenAI对象，使用"gpt-3.5-turbo-1106"模型
chat = ChatOpenAI(model="gpt-3.5-turbo-1106").bind(
    response_format={"type": "json_object"}
)

# 调用chat对象的invoke方法，传入包含SystemMessage和HumanMessage的列表作为参数
output = chat.invoke(
    [
        SystemMessage(
            content="Extract the 'name' and 'origin' of any companies mentioned in the following statement. Return a JSON list."
        ),
        HumanMessage(
            content="Google was founded in the USA, while Deepmind was founded in the UK"
        ),
    ]
)
# 打印输出内容
print(output.content)

In [ ]:
import json

# 使用json.loads()函数将output.content解析为JSON格式的数据
json.loads(output.content)

## [系统指纹](https://platform.openai.com/docs/guides/text-generation/reproducible-outputs)

OpenAI有时会以影响输出的方式更改模型配置。每当发生这种情况时，与生成相关联的系统指纹将会改变。

In [ ]:


# 创建ChatOpenAI对象，指定模型为"gpt-3.5-turbo-1106"
chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

# 调用generate方法生成对话
output = chat.generate(
    [
        [
            SystemMessage(
                content="Extract the 'name' and 'origin' of any companies mentioned in the following statement. Return a JSON list."
            ),
            HumanMessage(
                content="Google was founded in the USA, while Deepmind was founded in the UK"
            ),
        ]
    ]
)

# 打印生成的输出
print(output.llm_output)


## Azure类的重大变化

OpenAI V1重写了他们的客户端，并将Azure和OpenAI客户端分开。这导致在使用OpenAI V1时，LangChain接口发生了一些变化。

重大变化：
- 要在OpenAI V1中使用Azure嵌入，您需要使用新的`AzureOpenAIEmbeddings`，而不是现有的`OpenAIEmbeddings`。在使用Azure与`openai<1`时，`OpenAIEmbeddings`仍然有效。
```python
from langchain_openai import AzureOpenAIEmbeddings
```


推荐更改：
- 当使用`AzureChatOpenAI`或`AzureOpenAI`时，如果传入Azure端点（例如https://example-resource.azure.openai.com/），应该通过`azure_endpoint`参数或`AZURE_OPENAI_ENDPOINT`来指定。目前我们仍然通过`openai_api_base`/`base_url`或环境变量`OPENAI_API_BASE`来指定这一点，但不应该依赖于此。
- 当使用Azure聊天或嵌入模型时，通过`openai_api_key`参数或`AZURE_OPENAI_API_KEY`参数传入API密钥。目前我们仍然通过`OPENAI_API_KEY`来指定这一点，但不应该依赖于此。


## 工具

使用工具进行并行函数调用。

In [3]:
from typing import Literal

from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class GetCurrentWeather(BaseModel):
    """Get the current weather in a location."""

    location: str = Field(description="The city and state, e.g. San Francisco, CA")
    unit: Literal["celsius", "fahrenheit"] = Field(
        default="fahrenheit", description="The temperature unit, default to fahrenheit"
    )


prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant"), ("user", "{input}")]
)
model = ChatOpenAI(model="gpt-3.5-turbo-1106").bind(
    tools=[convert_pydantic_to_openai_tool(GetCurrentWeather)]
)
chain = prompt | model | PydanticToolsParser(tools=[GetCurrentWeather])

chain.invoke({"input": "what's the weather in NYC, LA, and SF"})

[GetCurrentWeather(location='New York, NY', unit='fahrenheit'),
 GetCurrentWeather(location='Los Angeles, CA', unit='fahrenheit'),
 GetCurrentWeather(location='San Francisco, CA', unit='fahrenheit')]